Installing packages...

In [5]:
!python --version

Python 3.9.13


In [91]:
# !pip install langchain==0.0.230
# !pip install transformers
# !pip install faiss-gpu

Importing libraries...

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import os

## Document loading...

In [2]:
loader = CSVLoader(file_path = 'anvisa_dataset.csv', 
                   csv_args = {'delimiter': '\t'})
data = loader.load()
print(len(data))

1238


In [3]:
print(data[0])

page_content='Empresa: Desconhecida - CNPJ: Desconhecido\nProdutoLote: GEL ANTISSÉPTICO ASSEPTGEL CRISTAL 100 ML (Todos);\nTipoProduto: Cosmético\nExpedienteNo: 3560796/20-3\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Apreensão, Proibição - Comercialização, Distribuição, Fabricação, Propaganda, Uso\nMotivacao: Considerando que a empresa Lima & Pergher Indústria e Comércio S/A, CNPJ 22.685.341/0001-80, não reconhece a fabricação do produto em embalagem de 100 mL, se tratando, portanto, de falsificação, e se tratando também de produto sem registro e fabricado por empresa desconhecida infringindo os arts. 2º e 12 da Lei 6360, de 23 de setembro de 1976 e tendo em vista o previsto nos arts 6º, 7º e inciso I do art. 67 da Lei 6360, de 23 de setembro de 1976.' metadata={'source': 'anvisa_dataset.csv', 'row': 0}


In [4]:
content = data[0].page_content
content

'Empresa: Desconhecida - CNPJ: Desconhecido\nProdutoLote: GEL ANTISSÉPTICO ASSEPTGEL CRISTAL 100 ML (Todos);\nTipoProduto: Cosmético\nExpedienteNo: 3560796/20-3\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Apreensão, Proibição - Comercialização, Distribuição, Fabricação, Propaganda, Uso\nMotivacao: Considerando que a empresa Lima & Pergher Indústria e Comércio S/A, CNPJ 22.685.341/0001-80, não reconhece a fabricação do produto em embalagem de 100 mL, se tratando, portanto, de falsificação, e se tratando também de produto sem registro e fabricado por empresa desconhecida infringindo os arts. 2º e 12 da Lei 6360, de 23 de setembro de 1976 e tendo em vista o previsto nos arts 6º, 7º e inciso I do art. 67 da Lei 6360, de 23 de setembro de 1976.'

## Loading language model to generate embeddings for each item in our dataset

In [5]:
model_name = "pucpr/biobertpt-all"
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(
    model_name=model_name, 
    model_kwargs=model_kwargs
)

No sentence-transformers model found with name /home/dvianna/.cache/torch/sentence_transformers/pucpr_biobertpt-all. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/dvianna/.cache/torch/sentence_transformers/pucpr_biobertpt-all were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequence

## Retrieval - FAISS

In [14]:
# Check if FAISS index exists. If not, create one
if os.path.exists('vectorstore/faiss/'):
    index = FAISS.load_local(folder_path='vectorstore/faiss/', embeddings=embedding)
else:
    # create faiss index
    index = FAISS.from_documents(data, embedding)   
    # Save the index to be loaded in the future
    index.save_local(folder_path='vectorstore/faiss/')

In [15]:
# Quey
query = 'European Directorate for the Quality of Medicines & HealthCare'

In [16]:
# based on relevance
index.similarity_search(query, k=3)

[Document(page_content='Empresa: 3.  SMS PHARMACEUTICALS LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 2515132/19-1\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Suspensão - Distribuição, Importação, Uso\nMotivacao: Considerando a suspensão pelo European Directorate for the Quality of Medicines & HealthCare - EDQM do certificado de adequabilidade em razão da presença da impureza NDMA, para o insumo Cloridrato de Ranitidina, fabricado pela empresa SMS Pharmaceuticals Limited. UNIT VII, localizada no Endereço Poosapatirega Mandal, Vizianagaram District India - 535 204 Kandivalasa Village, Andhra Pradesh, Índia.', metadata={'source': 'anvisa_dataset.csv', 'row': 682}),
 Document(page_content='Empresa: 3.  SMS PHARMACEUTICALS LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 2515132/19-1\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizaca

In [17]:
# based on relevance and diversity
index.max_marginal_relevance_search(query, k=3)

[Document(page_content='Empresa: 1.  SMS PHARMACEUTICALS LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 3364435/19-7\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Suspensão - Comercialização, Distribuição, Importação, Uso\nMotivacao: Considerando a suspensão pelo European Directorate for the Quality of Medicines & HealthCare - EDQM do certificado de adequabilidade em razão da presença da impureza NDMA, para o insumo Cloridrato de Ranitidina, fabricado pela empresa SMS Pharmaceuticals Limited - Índia.', metadata={'source': 'anvisa_dataset.csv', 'row': 5}),
 Document(page_content="Empresa: 1.  DR. REDDY´S LABORATORIES LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 2515189/19-4\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Suspensão - Comercialização, Distribuição, Importação, Uso\nMotivacao: Considerando a suspensão pelo Europ

In [18]:
index.similarity_search_with_score(query, k=3)

[(Document(page_content='Empresa: 3.  SMS PHARMACEUTICALS LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 2515132/19-1\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\nAcaoFiscalizacao: Suspensão - Distribuição, Importação, Uso\nMotivacao: Considerando a suspensão pelo European Directorate for the Quality of Medicines & HealthCare - EDQM do certificado de adequabilidade em razão da presença da impureza NDMA, para o insumo Cloridrato de Ranitidina, fabricado pela empresa SMS Pharmaceuticals Limited. UNIT VII, localizada no Endereço Poosapatirega Mandal, Vizianagaram District India - 535 204 Kandivalasa Village, Andhra Pradesh, Índia.', metadata={'source': 'anvisa_dataset.csv', 'row': 682}),
  45.944027),
 (Document(page_content='Empresa: 3.  SMS PHARMACEUTICALS LIMITED\nProdutoLote: \nTipoProduto: Insumo Farmacêutico\nExpedienteNo: 2515132/19-1\nAssunto:: 70351 - MEDIDA PREVENTIVA- Ações de Fiscalização em Vigilância Sanitária\

## Retrieval - Contextual Compression

In [19]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain import HuggingFacePipeline

In [20]:
model_id = "bigscience/bloomz-1b7"
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 512}
)

Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


In [21]:
compressor = LLMChainExtractor.from_llm(llm)

In [22]:
# Retrieval + document compression
# After retrieving the relevant documents, those documents are compressed. It means that only the relevant 
# information of each document, regarding the query context, is returned. 
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=index.as_retriever(k=3)
)

In [23]:
query = 'European Directorate for the Quality of Medicines & HealthCare'
compressed_docs = retriever.get_relevant_documents(query)

/home/dvianna/anaconda3/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/dvianna/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [24]:
for i, doc in enumerate(compressed_docs):
    print('%d: %s' % (i, doc.page_content))

0: *AS IS*
1: *AS IS*
2: *AS IS*
3: Suspensão
